In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, create_sql_agent, get_all_tool_names
from langchain.chains import SQLDatabaseChain
from langchain import OpenAI, SerpAPIWrapper, LLMChain, GoogleSearchAPIWrapper
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain
from langchain.utilities import PythonREPL

from chains.co2_sql import get_en_co2_sql_chain
from chains.weight_est import get_en_weight_est

In [8]:
search_chain = GoogleSearchAPIWrapper(k=10, search_engine="google")
math_chain = LLMMathChain(llm=ChatOpenAI(temperature=0)) # type: ignore
sql_chain = get_en_co2_sql_chain()
weight_est_chain = get_en_weight_est()


tools = [
    Tool(
        name="Weight estimator",
        func=weight_est_chain.run,
        description="Useful for estimating the weight of each item by passing all ingredients at once as input.",
    ),
    Tool(
        name="CO2 emission estimator",
        func=sql_chain.run,
        description="Useful for finding out the CO2 emission of each item by passing all ingredients at once as input.",
    ),
    Tool(
        name="Google search",
        func=search_chain.run,
        description="Useful for finding out the kg CO2e / kg if the ingredient is not in the database. Should only use if the ingredient weights more than 100 g.",
    ),
    Tool(
        name="Math calculator",
        func=math_chain.run,
        description="""Useful for calculating the CO2 emission of each ingredients by kg * kg CO2e / kg. You can for example ask: What is 0.1 * 1.26 + 0.2 * 0.5?"""
    )
]

In [9]:
prefix = """Act as a CO2 emission calculator that understands danish and english. 

Given a list of ingredients to a recipe, your job is to estimate the CO2 emission for each ingredient in kg CO2e and the recipe in total. 
It is important that you include the intermediate calculations for each ingredients and the total CO2 emission of the recipe 
in the final answer.
 
You have access to the following tools:"""

In [ ]:
# Remember, that your Final Answer cannot be trusted unless you show the intermediate calculations and the total CO2 emission of the recipe.

In [34]:
prefix = """Act as a CO2 emission calculator that understands danish and english. 

Given a list of ingredients to a recipe, your job is to estimate the CO2 emission for each ingredient in kg CO2e and the recipe in total. 
 
You have access to the following tools:"""

suffix = """

You can use all tools as many times as you want.

You solve this task by: 
1. Estimating the weights of the ingredients in the recipe using the weight estimation tool.
2. Finding the CO2 emission of each ingredient in kg CO2e per kg using the CO2 emission estimator tool. 
   Remember querying all ingredients. That includes salt, sugar etc.
3. If necessary, use Google Search to find the CO2 emission of a ingredient if CO2 emission is not in the database
   and it weights more than 10 g. 
4. Calculate the CO2 emission in kg per ingredient and sum them to get the total CO2 emission of the recipe using the math calculator tool.
5. Provide the answer where you show the CO2 calculation for each ingredient and the total CO2 emission of the recipe. 

Small amounts of ingredients like salt, sugar etc. that weight less than 0.01 kg can be ignored.

You must provide the Final Answer in the following format: 
Total CO2 emission: Z Total kg CO2e
Calculation method per ingredient: X kg CO2e / kg * Y kg = Z kg CO2e
ingredient 1: X1 * Y1 = Z1 
ingredient 2: X2 * Y2 = Z2 
etc.

In the Question section, the ingredients are provided. 

Begin! 

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [35]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [36]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [37]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    answer = agent_executor.run(
        """
        150 g red lentils
        1 large onion, 1 pepper
        2 cubes of vegetable bouillon
        3 cloves garlic, pressed
        1 can of chopped tomatoes,
        1 tin of tomato concentrate (140 g),
        1 can of coconut milk (400 ml),
        1 tbsp. lemon juice,
        1 tsp. sugar,
        ½ teaspoon chili powder,
        ½ teaspoon ground cumin,
        Possibly. 1/4 tsp. smoked paprika,
        A large handful of fresh basil, chopped
        (or 2 tsp dried basil)
        """
    )
    
    print("Total Tokens:", cb.total_tokens)
    print("Total prompt tokens:", cb.prompt_tokens)
    
    



> Entering new AgentExecutor chain...
Thought: I need to estimate the weight of each ingredient and find the CO2 emission of each ingredient. 
Action: Weight estimator
Action Input: 150 g red lentils, 1 large onion, 1 pepper, 2 cubes of vegetable bouillon, 3 cloves garlic, pressed, 1 can of chopped tomatoes, 1 tin of tomato concentrate (140 g), 1 can of coconut milk (400 ml), 1 tbsp. lemon juice, 1 tsp. sugar, ½ teaspoon chili powder, ½ teaspoon ground cumin, Possibly. 1/4 tsp. smoked paprika, A large handful of fresh basil, chopped (or 2 tsp dried basil)
Observation: Answer:
150 g red lentils: 0.15 kg
1 large onion: 0.285 kg
1 pepper: 0.150 kg
2 cubes of vegetable bouillon: 0.008 kg (2 x 4 g)
3 cloves garlic, pressed: negligible 
1 can of chopped tomatoes: 0.4 kg
1 tin of tomato concentrate (140 g): 0.140 kg
1 can of coconut milk (400 ml): 0.400 kg
1 tbsp. lemon juice: 0.015 kg
1 tsp. sugar: 0.005 kg
½ teaspoon chili powder: negligible 
½ teaspoon ground cumin: negligible 
Possibly.

In [38]:
print(answer)

Total CO2 emission: 2.95444 kg CO2e
Calculation method per ingredient: X kg CO2e / kg * Y kg = Z kg CO2e
Red lentils: 1.78 * 0.15 = 0.267 kg CO2e
Onion: 0.9 * 0.285 = 0.256 kg CO2e
Pepper: 1.02 * 0.15 = 0.153 kg CO2e
Vegetable bouillon: 0.38 * 0.008 = 0.003 kg CO2e
Chopped tomatoes: 1.26 * 0.4 = 0.504 kg CO2e
Tomato concentrate: 2.48 * 0.14 = 0.348 kg CO2e
Coconut milk: 3.5 * 0.4 = 1.4 kg CO2e
Lemon juice: 0.94 * 0.015 = 0.014 kg CO2e
Sugar: 1.92 * 0.005 = 0.009 kg CO2e


In [ ]:
"""
Example of Final Answer:
Calculation: X kg CO2e / kg * Y kg = Z kg CO2e
------------------------------------------------------------
2 carrots: 0.25 * 0.2 = 0.05
1 broccoli: 0.84 * 0.5 = 0.42
250 g minced chicken: 3.76 * 0.25 = 0.94
salt: Weight 0.001 kg is neglible
pepper: Weight 0.001 kg is neglible
------------------------------------------------------------
Total CO2 emission: 1.41 kg CO2e
------------------------------------------------------------
"""